**Run the following two cells before you begin.**

In [33]:
%autosave 10

Autosaving every 10 seconds


In [34]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [35]:
# Import the data set
df = pd.read_csv('cleaned_data.csv')

In [36]:
df.head(5)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


In [37]:
# Define the sigmoid function
def sigmoid(X):
    Y = 1 / (1 + np.exp(-X))
    return Y

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
# Create a train/test split
X_train, X_test, y_train, y_test = train_test_split(
df[['PAY_1', 'LIMIT_BAL']].values, df['default payment next month'].values,
test_size=0.2, random_state=24)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [40]:
from sklearn.linear_model import LogisticRegression

In [41]:
model_lr = LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [42]:
# Fit the logistic regression model on training data
model_lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
# Make predictions using `.predict()`
y_pred = model_lr.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [44]:
# Find class probabilities using `.predict_proba()`
y_pred_proba = model_lr.predict_proba(X_test)
y_pred_proba

array([[0.74826924, 0.25173076],
       [0.584297  , 0.415703  ],
       [0.79604453, 0.20395547],
       ...,
       [0.584297  , 0.415703  ],
       [0.82721498, 0.17278502],
       [0.66393435, 0.33606565]])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [47]:
# Add column of 1s to features
feature_one = np.hstack([np.ones((X_test.shape[0],1)), X_test])
feature_one

array([[ 1.0e+00,  2.0e+00,  1.6e+05],
       [ 1.0e+00,  1.0e+00,  5.0e+04],
       [ 1.0e+00, -1.0e+00,  2.0e+05],
       ...,
       [ 1.0e+00, -1.0e+00,  5.0e+04],
       [ 1.0e+00,  1.0e+00,  2.3e+05],
       [ 1.0e+00,  2.0e+00,  1.0e+05]])

In [49]:
# Get coefficients and intercepts from trained model
interceptcoef = np.concatenate([model_lr.intercept_.reshape(1,1), model_lr.coef_], axis=1)
interceptcoef

array([[-6.57647457e-11,  8.27451187e-11, -6.80876727e-06]])

In [50]:
# Manually calculate predicted probabilities
X_2 = np.dot(interceptcoef, np.transpose(feature_one))
y_pred_proba_2 = sigmoid(X_2)
y_pred_2 = y_pred_proba_2 >= 0.5

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [59]:
# Manually calculate predicted classes
y_pred_proba

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [58]:
# Compare to scikit-learn's predicted classes
y_pred_proba = model_lr.predict(X_test)
y_pred_proba

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

#### The results are same in this case.

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [51]:
from sklearn.metrics import roc_auc_score

In [52]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
roc_auc_score(y_test, y_pred_proba[:,1])

0.627207450280691

In [53]:
# Use manually calculated predicted probabilities to calculate ROC AUC
roc_auc_score(y_test, y_pred_proba_2.reshape(y_pred_proba_2.shape[1],))

0.627207450280691

#### Both the ROC AUC Scores are same..